<a href="https://colab.research.google.com/github/Muppidathe/deep_learning/blob/main/vgg16transferlearning_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision.models import vgg16
model=vgg16(pretrained=True)
model

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 153MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
from torchsummary import summary
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

# preparing data

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alsaniipe/flowers-dataset")

print("Path to dataset files:", path)

100%|██████████| 60.5M/60.5M [00:02<00:00, 26.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alsaniipe/flowers-dataset/versions/1


In [5]:
! cp -r $path ./ >>/dev/null

In [6]:
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from PIL import Image
import os
class flowerdataset(Dataset):
  def __init__(self,img_dir,transformer=None):
    self.img_dir=img_dir
    self.transformer=transformer
    self.labels=[]
    self.class_names={}
    self.img_paths=[]
    for label,class_name in enumerate(os.listdir(img_dir)):
      self.class_names[label]=class_name
      cls_dir=os.path.join(img_dir,class_name)
      for img_name in os.listdir(cls_dir):
        if img_name.split('.')[-1] in ['jpeg','jpg']:
          self.labels.append(label)
          image_path=os.path.join(cls_dir,img_name)
          self.img_paths.append(image_path)
  def __len__(self):
    return len(self.img_paths)
  def __getitem__(self,idx):
    label=self.labels[idx]
    image=self.img_paths[idx]
    if self.transformer:
      image=Image.open(image).convert('RGB')
      image=self.transformer(image)
    return image,label
transformer={'train':transforms.Compose([transforms.RandomResizedCrop(224),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[.485,.456,.406],std=[.229,.224,.225])]),
             'test':transforms.Compose([transforms.Resize(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[.485,.456,.406],std=[.229,.224,.225])])}

In [8]:
train_dataset=flowerdataset('/content/1/train',transformer['train'])
test_dataset=flowerdataset('/content/1/valid',transformer['test'])
train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=32,shuffle=True)

# freezing the model layer

In [9]:
import torch
output_classes_no=len(train_dataset.class_names)#2
for params in model.parameters():
  params.requires_grad=False
for params in model.classifier[-1].parameters():
  params.requires_grad=True
in_feature=model.classifier[-1].in_features
model.classifier[-1]=torch.nn.Linear(in_feature,output_classes_no)

In [13]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [14]:
from torch import nn
from torch import optim
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=.001)

In [15]:
epochs=10
for epoch in range(1,epochs+1):
  model.train()
  train_loss=[]
  test_loss=[]
  train_accuracy=[]
  test_accuracy=[]

  for img,label in train_dataloader:
    img=img.to(device)
    label=label.to(device)
    optimizer.zero_grad()
    pred=model(img)
    loss=criterion(pred,label)
    loss.backward()
    optimizer.step()
    #train loss
    train_loss.append(loss.item())
    #train accuracy
    _,predict=torch.max(pred,1)
    correct=(predict==label).sum().item()
    total=label.size(0)
    train_accuracy.append(correct/total)
  model.eval()
  with torch.no_grad():
    for img,label in test_dataloader:
      img=img.to(device)
      label=label.to(device)
      pred=model(img)
      loss=criterion(pred,label)
      #train loss
      test_loss.append(loss.item())
      #train accuracy
      _,predict=torch.max(pred,1)
      correct=(predict==label).sum().item()
      total=label.size(0)
      test_accuracy.append(correct/total)
  print(f'Epoch [{epoch}/{epochs}] --> train loss: {sum(train_loss)/len(train_loss)} train Accuracy: {sum(train_accuracy)/len(train_accuracy)} test loss: {sum(test_loss)/len(test_loss)} test Accuracy: {sum(test_accuracy)/len(test_accuracy)}')


Epoch [1/10] --> train loss: 0.296254288777709 train Accuracy: 0.8658275462962963 test loss: 0.17214594191561142 test Accuracy: 0.9331597222222222
Epoch [2/10] --> train loss: 0.1954872602596879 train Accuracy: 0.920167824074074 test loss: 0.18262259537974992 test Accuracy: 0.90625
Epoch [3/10] --> train loss: 0.1841515283100307 train Accuracy: 0.9222222222222222 test loss: 0.17441329018523297 test Accuracy: 0.9262152777777778
Epoch [4/10] --> train loss: 0.18882122868672013 train Accuracy: 0.9200231481481481 test loss: 0.1874103962133328 test Accuracy: 0.9201388888888888
Epoch [5/10] --> train loss: 0.17437837840989232 train Accuracy: 0.9317418981481481 test loss: 0.1627163471033176 test Accuracy: 0.9375
Epoch [6/10] --> train loss: 0.1586717368569225 train Accuracy: 0.9350115740740741 test loss: 0.18815923482179642 test Accuracy: 0.9192708333333334
Epoch [7/10] --> train loss: 0.19024826525710523 train Accuracy: 0.9161168981481481 test loss: 0.16001932322978973 test Accuracy: 0.94010